In [1]:
import os
# os.environ[“CUDA_DEVICE_ORDER”]=“PCI_BUS_ID”
os.environ["CUDA_VISIBLE_DEVICES"]="1"
# experiment_name = "3_heads_lr3_keep_op_identity+agenda_enriched_all+lr_e3+mult_scalar_per_action+glove"
# experiment_name = "crappy-red-dhole"

from allennlp.models.archival import Archive
import torch
from allennlp.common import Params
from allennlp.models.model import Model
from allennlp.common.params import with_fallback
from dataset_readers.spider_ratsql import SpiderRatsqlDatasetReader
# from models.semantic_parsing.spider_parser import SpiderParser
reader = SpiderRatsqlDatasetReader(tables_file="dataset/tables.json",max_instances=10)
# settings = Params.from_file(f"experiments/{experiment_name}/config.json")
# model = Model.load(config=settings, serialization_dir=f"experiments/{experiment_name}")


train_dataset = reader.read("dataset/train_spider.json")

Skipping writing to data cache since max_instances was specified.


In [2]:
# train_dataset = reader.read("dataset/train_spider.json")
from models.semantic_parsing.ratsql_encoder import RatsqlEncoder
from models.semantic_parsing.spider_decoder import SpiderParser
from allennlp.modules.seq2vec_encoders.boe_encoder import BagOfEmbeddingsEncoder

from allennlp.modules.attention import DotProductAttention
from allennlp.nn.beam_search import BeamSearch
from allennlp.modules.seq2seq_encoders.pass_through_encoder import PassThroughEncoder

from allennlp.data.vocabulary import Vocabulary
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
# from allennlp.data.iterators import BasicIterator
import torch.optim as optim
from allennlp.training.trainer import Trainer
import torch
# with torch.cuda.device(0):
vocab = Vocabulary.from_instances(train_dataset)
EMBEDDING_DIM = 200
HIDDEN_DIM = 200
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})

beam = BeamSearch(end_index=0,beam_size=10)

schema_encoder = RatsqlEncoder(encoder=PassThroughEncoder(200),entity_encoder=BagOfEmbeddingsEncoder(200),question_embedder=word_embeddings,action_embedding_dim=200)
model = SpiderParser(vocab=vocab,schema_encoder=schema_encoder, 
                     decoder_beam_search=beam,input_attention=DotProductAttention(),past_attention=DotProductAttention(),max_decoding_steps=10)


    # model.cuda()
#     iterator = BasicIterator(batch_size=15)
#     optimizer = optim.Adam(model.parameters(), lr=0.001)
#     iterator.index_with(vocab)
#     trainer = Trainer(model=model,
#                       optimizer=optimizer,
#                       iterator=iterator,
#                       train_dataset=train_dataset,
#                       validation_dataset=train_dataset,
#                       patience=10,
#                       num_epochs=1)
#     trainer.train()

In [3]:
for inst in train_dataset:
    a = inst
    res_list = model.forward_on_instances([inst])
    

[(0, tensor([-0.7761, -0.6166]), tensor([-0.7761, -0.6166]), tensor([[ 2.3778e-02, -7.9496e-02, -8.0419e-02,  4.4605e-02,  6.5860e-02,
          1.1118e-01,  7.0348e-03,  4.3595e-03,  1.1482e-01, -1.1954e-01,
          9.6892e-02, -7.1422e-02, -1.0083e-01,  1.1329e-01,  1.0542e-01,
          1.1554e-01,  5.9911e-02, -4.2190e-02,  1.0025e-01, -1.2284e-01,
          6.2961e-02,  6.5604e-02, -1.3144e-01, -1.8682e-02, -1.1196e-01,
          1.3401e-01,  5.1819e-02, -4.2703e-02, -1.2707e-01,  5.7586e-02,
          6.5120e-02, -9.7560e-02, -2.2562e-02, -9.4718e-02, -1.1738e-01,
         -9.9130e-02, -5.0624e-02, -1.2366e-01,  5.6155e-02,  4.0395e-02,
          1.0232e-02,  1.1044e-01, -2.1266e-02, -2.1174e-02, -8.0031e-02,
         -4.1267e-02, -4.7267e-02,  2.3135e-02,  1.2618e-01,  1.1439e-01,
          5.6238e-02,  6.2583e-02,  1.3562e-01, -5.0921e-02,  6.8498e-02,
          1.1983e-01,  1.1187e-01,  1.0440e-01,  7.8920e-02,  6.8454e-02,
          4.7484e-03,  8.9656e-03, -6.9821e-02, -8.

ValueError: too many values to unpack (expected 5)

In [ ]:
model.forward_on_instances([a])

In [5]:
import random

i=random.randint(0,len(train_dataset))
a=None
for inst in train_dataset[i:i+1]:
    a = inst
    res_list = model.forward_on_instances([inst])
    

In [5]:
a.fields['desc'].metadata

{'raw_question': 'What are the names of the states where at least 3 heads were born?',
 'question': ['What',
  'are',
  'the',
  'names',
  'of',
  'the',
  'states',
  'where',
  'at',
  'least',
  '3',
  'heads',
  'were',
  'born',
  '?'],
 'question_for_copying': ['What',
  'are',
  'the',
  'names',
  'of',
  'the',
  'states',
  'where',
  'at',
  'least',
  '3',
  'heads',
  'were',
  'born',
  '?'],
 'db_id': 'department_management',
 'sc_link': {'q_col_match': {'13,9': 'CPM'}, 'q_tab_match': {}},
 'cv_link': {'num_date_match': {'10,1': 'NUMBER',
   '10,4': 'NUMBER',
   '10,5': 'NUMBER',
   '10,6': 'NUMBER',
   '10,7': 'NUMBER',
   '10,10': 'NUMBER',
   '10,11': 'NUMBER',
   '10,12': 'NUMBER'},
  'cell_match': {}},
 'columns': [['<type: text>', '*', '<table-sep>', '<any-table>'],
  ['<type: number>', 'department', 'id', '<table-sep>', 'department'],
  ['<type: text>', 'name', '<table-sep>', 'department'],
  ['<type: text>', 'creation', '<table-sep>', 'department'],
  ['<type: n

In [6]:
b= res_list[0]['initial_state']
    

In [5]:
c= b.get_valid_actions()

In [ ]:
# c.get_valid_actions()
b.possible_actions

In [ ]:
len(train_dataset)

In [5]:
import random
i=random.randint(0,len(train_dataset))
a=None
for inst in train_dataset[i:i+1]:
#     print(inst)
    a=inst
# b=a.fields['relation']
# b.array
b=a.fields['world']
c=a.fields['desc']
d=a.fields['item'] 
e = a.fields["schema"]
# b
# print(a.fields['utterance'])
for entity in b.metadata.db_context.knowledge_graph.entities:
    print(entity)

column:foreign:management:department_id
column:foreign:management:head_id
column:number:department:budget_in_billions
column:number:department:num_employees
column:number:department:ranking
column:number:head:age
column:primary:department:department_id
column:primary:head:head_id
column:text:department:creation
column:text:department:name
column:text:head:born_state
column:text:head:name
column:text:management:temporary_acting
string:california
string:state
table:department
table:head
table:management


In [6]:
for entity in (c.metadata['columns']+c.metadata['tables']):
def normalize_schema_constant(entity):
#     print(entity)
    col = "_".join(entity)
    col =  col.split("_<table-sep>_")
    if len(col)==1:
        return "_".join(entity)
#         print()
    else:
        table = col[1]
        col = col[0].split(">_")[1]
        return f"{table}@{col}"
#         print()
        
    

['<type: text>', '*', '<table-sep>', '<any-table>']
<any-table>@*
['<type: number>', 'department', 'id', '<table-sep>', 'department']
department@department_id
['<type: text>', 'name', '<table-sep>', 'department']
department@name
['<type: text>', 'creation', '<table-sep>', 'department']
department@creation
['<type: number>', 'ranking', '<table-sep>', 'department']
department@ranking
['<type: number>', 'budget', 'in', 'billions', '<table-sep>', 'department']
department@budget_in_billions
['<type: number>', 'num', 'employees', '<table-sep>', 'department']
department@num_employees
['<type: number>', 'head', 'id', '<table-sep>', 'head']
head@head_id
['<type: text>', 'name', '<table-sep>', 'head']
head@name
['<type: text>', 'born', 'state', '<table-sep>', 'head']
head@born_state
['<type: number>', 'age', '<table-sep>', 'head']
head@age
['<type: number>', 'department', 'id', '<table-sep>', 'management']
management@department_id
['<type: number>', 'head', 'id', '<table-sep>', 'management']
man

In [18]:
e.entity_texts

[[department, id],
 [head, id],
 [budget, in, billions],
 [num, employees],
 [ranking],
 [age],
 [department, id],
 [head, id],
 [creation],
 [name],
 [born, state],
 [name],
 [temporary, acting],
 [rank],
 [department],
 [head],
 [management]]

In [4]:
print(a.fields['valid_actions'])

ListField of 144 ProductionRuleFields : 
 	 ProductionRuleField with rule: arg_list -> [expr, ",", arg_list] (is_global_rule: True) in namespace: 'rule_labels'.' 
 	 ProductionRuleField with rule: arg_list -> [expr] (is_global_rule: True) in namespace: 'rule_labels'.' 
 	 ProductionRuleField with rule: arg_list_or_star -> ["*"] (is_global_rule: True) in namespace: 'rule_labels'.' 
 	 ProductionRuleField with rule: arg_list_or_star -> [arg_list] (is_global_rule: True) in namespace: 'rule_labels'.' 
 	 ProductionRuleField with rule: binaryop -> ["!="] (is_global_rule: True) in namespace: 'rule_labels'.' 
 	 ProductionRuleField with rule: binaryop -> ["*"] (is_global_rule: True) in namespace: 'rule_labels'.' 
 	 ProductionRuleField with rule: binaryop -> ["+"] (is_global_rule: True) in namespace: 'rule_labels'.' 
 	 ProductionRuleField with rule: binaryop -> ["-"] (is_global_rule: True) in namespace: 'rule_labels'.' 
 	 ProductionRuleField with rule: binaryop -> ["/"] (is_global_rule: Tru

In [13]:
d.metadata.schema.columns[3]

Column(id=3, table=Table(id=0, name=['city'], unsplit_name='city', orig_name='city', columns=[Column(id=1, table=..., name=['city', 'id'], unsplit_name='city id', orig_name='City_ID', type='number', foreign_key_for=None), Column(id=2, table=..., name=['official', 'name'], unsplit_name='official name', orig_name='Official_Name', type='text', foreign_key_for=None), ..., Column(id=4, table=..., name=['area', 'km', '2'], unsplit_name='area km 2', orig_name='Area_km_2', type='number', foreign_key_for=None), Column(id=5, table=..., name=['population'], unsplit_name='population', orig_name='Population', type='number', foreign_key_for=None), Column(id=6, table=..., name=['census', 'ranking'], unsplit_name='census ranking', orig_name='Census_Ranking', type='text', foreign_key_for=None)], primary_keys=[Column(id=1, table=..., name=['city', 'id'], unsplit_name='city id', orig_name='City_ID', type='number', foreign_key_for=None)]), name=['status'], unsplit_name='status', orig_name='Status', type='

In [12]:
"asdas_adsdas".split()

['asdas_adsdas']

In [10]:
# print()


[['city'], ['farm'], ['farm', 'competition'], ['competition', 'record']]

In [29]:

# print(b)


In [2]:
import json
from dataset_readers.ratsql_spider_bert import *

# builder = RelationBuilder()


db_path = "dataset/database"
pre = SpiderEncoderV2Preproc()
schemas, eval_foreign_key_maps = pre.load_tables(["dataset/tables.json"])
# for path in paths:
for db_id, schema in tqdm(schemas.items(), desc="DB connections"):
    sqlite_path = Path(db_path) / db_id / f"{db_id}.sqlite"
    source: sqlite3.Connection
    with sqlite3.connect(sqlite_path) as source:
        dest = sqlite3.connect(':memory:')
        dest.row_factory = sqlite3.Row
        source.backup(dest)
    schema.connection = dest
    
examples=[]
raw_data = json.load(open("dataset/train_spider.json"))
for entry in raw_data:
    item = SpiderItem(
        text=entry['question_toks'],
        code=entry['sql'],
        schema=schemas[entry['db_id']],
        orig=entry,
        orig_schema=schemas[entry['db_id']].orig)
#     desc = pre.preprocess_item(item,"train")
    examples.append(item)

        # Backup in-memory copies of all the DBs and create the live connections


DB connections: 100%|██████████| 166/166 [00:00<00:00, 354.40it/s]


In [7]:
# examples
# 
import pandas as pd
print 
# 
import tqdm
for item in tqdm.tqdm(examples):
    desc = pre.preprocess_item(item,"train")
    q = desc['question']
    q_len = len(q)
    t = [x[0] for x in  desc['tables']]
    t_len = len(t)
    c = ["_".join(x) for x in  desc['columns']]
    c_len = len(c)
    enc = q+c+t
    print(enc)
    relation = pre.compute_relations(desc,len(enc),q_len,c_len,range(c_len+1),range(t_len+1))
#     with np.printoptions(threshold=np.inf):
#     print(np.array2string())

    arr = pd.DataFrame(relation)
#     print()
    break
# arr


  0%|          | 0/7000 [00:00<?, ?it/s]

['How', 'many', 'heads', 'of', 'the', 'departments', 'are', 'older', 'than', '56', '?', '<type: text>_*_<table-sep>_<any-table>', '<type: number>_department_id_<table-sep>_department', '<type: text>_name_<table-sep>_department', '<type: text>_creation_<table-sep>_department', '<type: number>_ranking_<table-sep>_department', '<type: number>_budget_in_billions_<table-sep>_department', '<type: number>_num_employees_<table-sep>_department', '<type: number>_head_id_<table-sep>_head', '<type: text>_name_<table-sep>_head', '<type: text>_born_state_<table-sep>_head', '<type: number>_age_<table-sep>_head', '<type: number>_department_id_<table-sep>_management', '<type: number>_head_id_<table-sep>_management', '<type: text>_temporary_acting_<table-sep>_management', 'department', 'head', 'management']


In [8]:
for v in a.items():
    print(v)

('utterance', <allennlp.data.fields.text_field.TextField object at 0x7f36567d44c0>)
('valid_actions', <allennlp.data.fields.list_field.ListField object at 0x7f3660a35fd0>)
('action_sequence', <allennlp.data.fields.list_field.ListField object at 0x7f36565e9730>)
('world', <allennlp.data.fields.metadata_field.MetadataField object at 0x7f36565e9d00>)
('schema', <dataset_readers.fields.spider_knowledge_graph_field.SpiderKnowledgeGraphField object at 0x7f36565e0be0>)


In [10]:
b=a.fields['utterance']

In [14]:
b.sequence_length()
b.tokens

In [20]:
c=a.fields['schema']
c.__dict__

[nltk_data] Downloading package stopwords to /home/ohadr/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
builder.compute_relations(dict(),)

In [10]:
arr.to_csv("t.csv")